In [145]:
# import
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from urllib.parse import quote_plus

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from pymongo import MongoClient

import re
import random

Header

In [146]:
# 페이지 추출함수
def pageExtraction(url1):
  webpage = requests.get(url1)
  soup = BeautifulSoup(webpage.content, "html.parser")
  # #논문 제목,링크 추출
  df_data1 = []
  df_name = []
  df_link = []
  df_link1 = []
  division = []
  data1s = soup.select('#main > div > div > div > a > div')
  data2s = soup.select('#main > div > div > div > a > h3 > div')

  for i, data in enumerate(data1s):
    if i%2 == 0:
      st = str(data1s[i]).find('>')
      end = str(data1s[i]).find('</')
      df_link1.append((str(data1s[i])[st+1:end]))

  for i, data in enumerate(data2s):
    st = str(data2s[i]).find('>')
    end = str(data2s[i]).find('</')
    df_name.append((str(data2s[i])[st+1:end-1]))

  #링크에서 외국어처리되는 부분 삭제
  for link in df_link1:
    if link.find('?')!=-1:
      df_link.append(link[:-6])
    else :
      df_link.append(link)

  # #특허번호와 국가코드 추출
  df_patent = []
  df_code = []
  if (df_link[0].find('www.google.com') != -1):
    for link in df_link:
      df_patent.append(link.replace('www.google.com/patents/',''))
      df_code.append(link.replace('www.google.com/patents/','')[0:3])
  elif (df_link[0].find('www.google.co.kr') != -1):
    for link in df_link:
      df_patent.append(link.replace('www.google.co.kr/patents/',''))
      df_code.append(link.replace('www.google.co.kr/patents/','')[0:3])
    #국가코드 3자리인 경우도 포함하기 위하여 3자리에서 숫자아닌경우 slicing
  for i,code in enumerate(df_code):
   if str.isdigit(code[2:3]):
      df_code[i]=(code[0:2])
   else :
      df_code[i]=(code[0:3])

  #for dataframe
  df = pd.DataFrame(index=range(0,len(df_data1)), columns=['특허제목', '링크', '특허번호','국가코드'])
  df['특허제목'] = df_name
  df['링크'] = df_link
  df['특허번호'] = df_patent
  df['국가코드'] = df_code
  return df

# 적용국가 추출함수
def application(url):
  data = requests.get(url)
  soup = BeautifulSoup(data.content, 'html.parser')  
  country = soup.find_all('span', attrs={'itemprop': 'countryCode'})
  Date = soup.find_all('span', attrs={'itemprop': 'filingDate'})
  nation_name = []
  nation_date = []
  # ----------------특허 적용 국가 리스트----------------
  for n in country:
       nation_name.append(n.get_text())
  country_name =  [] 
  for i in nation_name:
     country_name.append(i)
  for n in Date:
     nation_date.append(n.get_text())
  country_date =  [] 
  for i in nation_date:
      country_date.append(i)
  del country_date[0]

  new_contryName = []
  new_contryDate = []
  for i,v in enumerate(country_date):
    new_contryDate.append(v[:4])
  for i in range(len(new_contryDate)):
    new_contryName.append(country_name[-1-i])
  new_contryName.reverse()

  df_application = []
  for i in range(len(new_contryName)):
    df_application.append(new_contryName[i]+'('+new_contryDate[i]+')')

  df_output = []
  df_output.append(' / '.join(df_application))

  inventor = soup.find_all('dd',{'itemprop':'inventor'})
  assigneeCurrent = soup.find_all('dd',{'itemprop':'assigneeCurrent'})  
  patent_inventor = []
  patent_assignee = []

  for n in inventor:
      patent_inventor.append(n.get_text())

  df_patent_inventor = []
  for i in range(len(patent_inventor)):
    if i == 0:
      df_patent_inventor.append(patent_inventor[i])
    else :
      df_patent_inventor.append(' / '+patent_inventor[i])
  df_patent_inventor= "".join(df_patent_inventor)

  for n in assigneeCurrent:
      patent_assignee.append(n.get_text())
  df_patent_assignee = []
  for i in range(len(patent_assignee)):
    if i ==0:
      df_patent_assignee.append(patent_assignee[i])
    else :
      df_patent_assignee.append(' / '+patent_assignee[i])

  Str_assignee = "".join(df_patent_assignee)
  Str_assignee = Str_assignee.replace("\n","")
  Str_assignee = Str_assignee.lstrip()
  new_list_assignee = Str_assignee.split("   ")
  new_list_assignee= "".join(new_list_assignee)

  df1 = pd.DataFrame(index=range(0,1), columns=['적용국가','Inventor','assignee'])
  df1['적용국가'] = df_output
  df1['Inventor'] = df_patent_inventor
  df1['assignee'] = new_list_assignee
  return df1

def search(keywords, N):
  keywords.replace(' ','+')
  keyword = keywords.split(',')
  baseUr1 = 'https://www.google.com/search?q='

  #데이터 프레임 concat 기능
  whole_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
  plusUr1 = ''
  ur1 = ''
  for i in range(int(N/10)):
    for key in keyword:
      if(key != 'end'):
        plusUr1 = plusUr1 + '%22' + key +'%22+'
      else:
        plusUr1 = plusUr1[:-1]
    ur1 = baseUr1 + plusUr1 + '&tbm=pts&start=' + str(i*10)
    plusUr1 = ''
    df = pageExtraction(ur1)
    whole_df = pd.concat([whole_df,df], ignore_index=True)
  return keywords, whole_df

def search_advanced(df):
  df_app = []
  df2 = pd.DataFrame(columns=['적용국가','Inventor','assignee'])
  for link in df['링크']:
    df1 = application('http://'+link)
    df2 = pd.concat([df2, df1], ignore_index=True)

  df_out = pd.concat([df,df2],axis=1)
  return df_out

def field_search(df):
  index = df.columns.tolist()
  print('검색가능한 field들 :',index)
  index.append('end')

  df_output = df
  field = input('검색할 field를 입력하세요(끝내고싶으면 end 입력) : ')
  while(field != 'end'):
      while (field not in index):
        field = input('검색할 field를 다시 입력하세요(끝내고싶으면 end 입력) : ')
      key = input('검색할 키워드를 입력하세요 : ')
      df_output = df[df[field].str.contains(key)]
      df = df_output
      field = input('검색할 field를 입력하세요(끝내고싶으면 end 입력) : ')
    
  return df_output

def paper_citation(df):
  keyword = []
  keyword.append(input('특허번호 입력 :')) 
  str_keyword ="".join(keyword)
  keyowrd_url = 'https://patents.google.com/patent/'+ str_keyword

  data = requests.get(keyowrd_url)
  soup = BeautifulSoup(data.content, 'html.parser')
  country = soup.find_all('tr',{'itemprop':'backwardReferences'})
  information = []
  non_patent_citation = soup.find_all('tr',{'itemprop':'detailedNonPatentLiterature'}) 
  NPC = []

  for n in non_patent_citation:
       NPC.append(n.get_text())
  
  non_patent = "".join(NPC)
  non_patent = non_patent.replace("\n","")
  new_non_patent = non_patent.split("*")
  del new_non_patent[len(new_non_patent)-1]

  for n in country:
    information.append(n.get_text())

  indexlist = []
  for i,data in enumerate(information):
    index = 0
    while index > -1:
      index = data.find('\n',index)
      if index > -1:
        indexlist.append(index)
        index += len('\n')

  publication_number = []
  priority_date = []
  publication_date = []
  assignee = []
  title = []
  line = ''
  count = 0

  for i,index in enumerate(indexlist):
    if index == 0:
      line = information[count]
      publication_number.append(line[indexlist[i+2]+1:indexlist[i+3]])
      if line.find('*')==-1:
        priority_date.append(line[indexlist[i+6]+1:indexlist[i+7]])
        publication_date.append(line[indexlist[i+7]+1:indexlist[i+8]])
        assignee.append(line[indexlist[i+8]+1:indexlist[i+9]])
        title.append(line[indexlist[i+9]+1:indexlist[i+10]])
      else :
        priority_date.append(line[indexlist[i+7]+1:indexlist[i+8]])
        publication_date.append(line[indexlist[i+8]+1:indexlist[i+9]])
        assignee.append(line[indexlist[i+9]+1:indexlist[i+10]])
        title.append(line[indexlist[i+10]+1:indexlist[i+11]])
      count = count + 1
    


  df2 = pd.DataFrame(index=range(0,len(publication_number)), columns=['Publication number', 'Priority date', 'Publication date','Assignee','Title'])
  df2['Publication number'] = publication_number
  df2['Priority date'] = priority_date
  df2['Publication date'] = publication_date
  df2['Assignee'] = assignee
  df2['Title'] = title

  df3 = pd.DataFrame(index=range(0,len(new_non_patent)), columns=['Non-Patent Citations title'])
  df3['Non-Patent Citations title'] = new_non_patent
  return df2, df3


def Data_File(keyword,df):
  file_name = ''
  for i in range(len(keyword)):
    file_name = file_name + keyword[i] + ' '
  file_name = file_name+'.xlsx'
  excel_name = ''
  excel_name = file_name.replace('end','')
    # print(excel_name)
  df.to_excel(excel_name,index =False)
  
  
def Search_patent_news():
  #검색할 키워드 입력
  query = input('검색할 키워드를 입력하세요: ')


  #크롬드라이버로 원하는 url로 접속
  url = 'https://www.patentlyapple.com/patently-apple/'
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
  driver.get(url)
  time.sleep(3)

  #검색창에 키워드 입력 후 엔터

  search_box = driver.find_element_by_css_selector("#search-blog > div > input")
  search_box.send_keys(query)
  search_box.send_keys(Keys.RETURN)
  time.sleep(3)

  Apple_Patent_news = []
  Apple_Patent_link = []
  for k in range(0,100):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    Patent__apple_title = soup.find_all('h3', attrs={'class': 'entry-header font-entryheader'})
    for i in Patent__apple_title:
      Apple_Patent_news.append(i.get_text())

    Patent__apple_link = soup.select('h3 > a')
    for i in Patent__apple_link:
        Apple_Patent_link.append(i.attrs['href'])

    try:
      if(k==0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span').click()
      if(k > 0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span[3]/a').click()
    except NoSuchElementException :
      print("Maximum page is ", k+1)
      break
  time.sleep(2)
  driver.close()

  df = pd.DataFrame(index=range(0,len(Apple_Patent_news)), columns=['News_Title','News_Link'])
  df['News_Title'] = Apple_Patent_news
  df['News_Link'] = Apple_Patent_link

  return df

def search_details(wholeurl):
  driver = webdriver.Chrome('./chromedriver')
  patent_name = []
  inventor = []
  assignee = []
  summary = []

  # 첫 페이지에 접속
  tempurl = wholeurl.replace('&page=*','')
  driver.get(tempurl)
  time.sleep(2)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('#htmlContent')
  try:
    count = soup.select_one('#count > div.layout.horizontal.style-scope.search-results > span.flex.style-scope.search-results > span:nth-child(3)').text
  except:
    count = 1000
  count = count.replace(',','')

  # 논문제목, inventor, assignee추출
  for i, n in enumerate(notices):
      if i%4 == 0:
          patent_name.append(n.text.strip())
      elif i%4 == 1:
          inventor.append(n.text.strip())
      elif i%4 == 2:
          assignee.append(n.text.strip())
      else :
          summary.append(n.text.strip())
  # 적용국가, 특허코드 추출
  object = soup.select('#resultsContainer > section > search-result-item > article > div > div > div > div.flex.style-scope.search-result-item > h4.metadata.style-scope.search-result-item > span')
  ob = []
  for o in object:
      ob.append(o.text)  
  applied_country = ''
  country = []
  patent_code = []
  for o in ob:
      count1 = o.count('\n')
      if count1 == 0:
          applied_country = applied_country + o + ', '
      if count1 == 4:
          applied_country = applied_country[:-2]
          country.append(applied_country)
          applied_country = ''
          patent_code.append(o.replace('\n',''))

  # 검색결과가 1000개이상 나오지않아서 오래걸리는 것을 방지
  page = int(count)/10
  if page > 99:
      page = 99

  # 첫 페이지에서 확인한 정보로 나머지 페이지 확인
  for i in range(1,int(page)+1):
      tempurl = wholeurl.replace('&page=*','&page=')+f'{i}'
      driver.get(tempurl)
      rand_value = randint(1,3)
      time.sleep(rand_value)
      
      html = driver.page_source
      soup = BeautifulSoup(html, 'html.parser')
      notices = soup.select('#htmlContent')   
      
      object = soup.select('#resultsContainer > section > search-result-item > article > div > div > div > div.flex.style-scope.search-result-item > h4.metadata.style-scope.search-result-item > span')
      ob = []
      for o in object:
          ob.append(o.text)  

      applied_country = ''
      for o in ob:
          count = o.count('\n')
          if count == 0:
              applied_country = applied_country + o + ', '
          if count == 4:
              applied_country = applied_country[:-2]
              country.append(applied_country)
              applied_country = ''
              patent_code.append(o.replace('\n',''))

      for i, n in enumerate(notices):
          if i%4 == 0:
              patent_name.append(n.text.strip())
          elif i%4 == 1:
              inventor.append(n.text.strip())
          elif i%4 == 2:
              assignee.append(n.text.strip())
          else :
              summary.append(n.text.strip())

  driver.close()
  return patent_name, patent_code, inventor, assignee ,country

# 데이터 프레임 생성
def search_details_df(patent_name, patent_code, inventor, assignee ,country):
    output = pd.DataFrame({'특허제목':patent_name, '특허코드':patent_code, 'inventor':inventor, 'assignee':assignee, '적용국가':country})
    return output

# google patent 링크 생성함수 (en:english, kr:korean) 제목또한 해당 언어로 return
def google_link(list,language):
    base_link = 'https://patents.google.com/patent/'
    output_link = []
    for l in list:
        output_link.append(base_link+l+'/'+language)
    
    return output_link

def whole_name(link):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0', 'X-Requested-With': 'XMLHttpRequest'}  
    whole_names = []
    dates = []
    pdf_link = []
    patent_citation_num = []
    n1 = []
    n2 = []
    for l in link:
        data = requests.get(l, headers=headers)
        soup = BeautifulSoup(data.content, 'html.parser')
        try:
          title = soup.select_one('body > search-app > article > h1').text
        except:
          title = ' - no title - '
        try:
          date = soup.find(attrs = {'itemprop':'publicationDate'}).text
        except:
          date = '-'
        ptnum = soup.select('body > search-app > article > section > h2')
        
        st = title.find(' - ')
        end = title.rfind(' - ')
        whole_names.append(title[st+3:end-9])
        dates.append(date)
        link2 = soup.find('a')
        pdf_link.append(link2.attrs['href'])
        
        for p in ptnum:
            if p.text.find('Patent Citations ')==0:
                str1 = p.text
                n1 = re.findall('\(([^)]+)', str1)
            if p.text.find('Non-Patent Citations')==0:
                str2 = p.text
                n2 = re.findall('\(([^)]+)', str2)
            num = 0
            if n1:
                num = num + int(n1[0])
            if n2:
                num = num + int(n2[0])
        patent_citation_num.append(num)

    return whole_names, dates, pdf_link, patent_citation_num

def change_ptname(df):
    google_links = google_link(df['특허코드'],'en')
    whole_names, dates, pdf_link, patent_citation_num = whole_name(google_links)
    
    
    df['특허제목'] = whole_names
    df['google patent link'] = google_links
    df['출원날짜'] = dates
    df['pdf link'] = pdf_link
    df['인용횟수'] = patent_citation_num

    
    return df

def Search_patent_news():
  #검색할 키워드 입력
  query = input('검색할 키워드를 입력하세요: ')


  #크롬드라이버로 원하는 url로 접속
  driver = webdriver.Chrome('./chromedriver')
  url = 'https://www.patentlyapple.com/patently-apple/'
  driver.get(url)
  time.sleep(2)

  #검색창에 키워드 입력 후 엔터

  search_box = driver.find_element_by_css_selector("#search-blog > div > input")
  search_box.send_keys(query)
  search_box.send_keys(Keys.RETURN)
  time.sleep(2)

  Apple_Patent_news = []
  Apple_Patent_link = []
  for k in range(0,100):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    Patent__apple_title = soup.find_all('h3', attrs={'class': 'entry-header font-entryheader'})
    for i in Patent__apple_title:
      Apple_Patent_news.append(i.get_text())

    Patent__apple_link = soup.select('h3 > a')
    for i in Patent__apple_link:
        Apple_Patent_link.append(i.attrs['href'])

    try:
      if(k==0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span').click()
      if(k > 0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span[3]/a').click()
    except NoSuchElementException :
      print("Maximum page is ", k+1)
      break
  driver.close()

  df = pd.DataFrame(index=range(0,len(Apple_Patent_news)), columns=['News_Title','News_Link'])
  df['News_Title'] = Apple_Patent_news
  df['News_Link'] = Apple_Patent_link

  return df

def detail_search(keyword, inventor, assignee, country):
    baseurl = 'https://patents.google.com/?q='
    # wholeurl = baseurl + keyword + '&inventor=' + inventor + '&assignee=' + assignee + '&country=' + country + '&page=*'
    wholeurl = baseurl + keyword + '&inventor=' + inventor + '&assignee=' + assignee + '&page=*'
    
    patent_name, patent_code, inventor, assignee ,country = search_details(wholeurl)
    output = search_details_df(patent_name, patent_code, inventor, assignee ,country)
    output2 = change_ptname(output)
    
    return output2
  
# 제목과 링크 추출
def extractlink(url):
  # url = 'https://www.patentlyapple.com/patently-apple/autonomous-vehicle-technology/'
  driver = webdriver.Chrome('./chromedriver')
  driver.get(url)
  time.sleep(3)

  Apple_Patent_news = []
  Apple_Patent_link = []
  title_index=0
  for k in range(0,10):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    Patent__apple_title = soup.find_all('h2', attrs={'class': 'entry-header font-entryheader'})

    for i in Patent__apple_title:
      Apple_Patent_news.append(i.get_text()[2:-2])
    Patent__apple_link = soup.select('h2 > span > a')
    for i in Patent__apple_link:
        Apple_Patent_link.append(i.attrs['href'])

    try:
      if(len(Apple_Patent_link)==25*(k+1)):
        # page_str = 'https://www.patentlyapple.com/patently-apple/autonomous-vehicle-technology/' + 'page/'+ str(k+1) + '/'
        page_str = url + 'page/'+ str(k+1) + '/'
        driver.get(page_str)
      if(len(Apple_Patent_link)<25*(k+1)):
        raise Exception
    except Exception:
      print("Maximum page is ", k)
      break
  time.sleep(2)
  driver.close()

  df1 = pd.DataFrame(index=range(0,len(Apple_Patent_news)), columns=['News_Title','News_Link'])
  df1['News_Title'] = Apple_Patent_news
  df1['News_Link'] = Apple_Patent_link
  return df1

# 첫번째필터
def filter1(df):
    driver = webdriver.Chrome('./chromedriver')
    patents_list = []
    link_list = []
    for i in df.index:
        # 웹페이지 가져오기
        link = df['News_Link'][i]
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        contents = soup.select('#post-container > div.container > div.row.pagebody > div.col-sm-8.col-sm-push-4.col-md-6.col-md-push-3 > div > div > article > div.entry-more.font-entrybody > p')
        contents_list = []
        befor_list = patents_list[:]
        patents_list_un = []
        for c in contents:
            contents_list.append(c.text)
        # html의 한 구조체마다 반복문
        for j, con in enumerate(contents_list):
            # 첫번째 필터를 통해 데이터 추출
            c = ','
            lst = []
            # 구조체에서의 ,위치 return
            for pos,char in enumerate(con):
                if(char == c):
                    lst.append(pos)
            for i, l in enumerate(lst):
                before = 0
                now = l
                if i!=0:
                    before = lst[i-1]
                if (now - before) == 4:
                    patents_list_un.append(contents_list[j][before-3:now+4])
            
            # 두번째 필터를 통해 데이터 추출
            pos1 = con.find('20210')
            if pos1 != -1:
                patents_list.append(con[pos1:pos1+11])
            pos2 = con.find('20200')
            if pos2 != -1:
                patents_list.append(con[pos2:pos2+11])
            pos3 = con.find('20190')
            if pos3 != -1:
                patents_list.append(con[pos3:pos3+11])
            pos4 = con.find('20180')
            if pos4 != -1:
                patents_list.append(con[pos4:pos4+11])
      
        
        for code in patents_list_un:
            try:
                if ((code[0]==' ')&code[1:3].isdigit()&code[4:7].isdigit()&code[8:11].isdigit()):
                    # print(code[1:])
                    patents_list.append(code[1:].replace(',',''))
                # print(patents_list)
            except IndexError:
                a=0
        
        # 세번째 필터를 통해 데이터 추출
        Patent_title_number= soup.select('div > p > a')
        Patent_title_here = soup.select('div > p > span > a')
        
        for k in Patent_title_number:
            if k.attrs['href'].find('uspto') != -1:
                if k.find('patentnumber') != -1:
                    patents_list.append(k.attrs['href'].replace(',','')[-8:])
                else: 
                    patents_list.append(k.attrs['href'].replace(',','')[-11:])
        for k in Patent_title_here:
            if k.attrs['href'].find('uspto') != -1:
                if k.find('patentnumber') != -1:
                    patents_list.append(k.attrs['href'].replace(',','')[-8:])
                else: 
                    patents_list.append(k.attrs['href'].replace(',','')[-11:])
        
        if (len(befor_list) == len(patents_list)):
            link_list.append(link)
            # print(link)

    return patents_list

def applelink(codes):
    links = []
    for code in codes:
        links.append('https://patents.google.com/patent/US'+code)
    return links

def application2(url):
  data = requests.get(url)
  soup = BeautifulSoup(data.content, 'html.parser')
  country = soup.find_all('span', attrs={'itemprop': 'countryCode'})
  Date = soup.find_all('span', attrs={'itemprop': 'filingDate'})
  nation_name = []
  nation_date = []
  # ----------------특허 적용 국가 리스트----------------  
  for n in country:
    try:
      nation_name.append(n.get_text())
    except:
      nation_name.append('')
  country_name =  [] 
  for i in nation_name:
    try:
      country_name.append(i)
    except:
      country_name.append('')
  for n in Date:
    try:
      nation_date.append(n.get_text())
    except:
      nation_date.append('')
  country_date =  [] 
  for i in nation_date:
      country_date.append(i)
  
  new_contryName = []
  new_contryDate = []
  for i,v in enumerate(country_date):
    new_contryDate.append(v[:4])
  for i in range(len(new_contryDate)):
    try:
      new_contryName.append(country_name[-1-i])
    except:
      new_contryName.append('')
  new_contryName.reverse()

  df_application = []
  for i in range(len(new_contryName)):
    df_application.append(new_contryName[i]+'('+new_contryDate[i]+')')

  df_output = []
  df_output.append(' / '.join(df_application))

  inventor = soup.find_all('dd',{'itemprop':'inventor'})
  assigneeCurrent = soup.find_all('dd',{'itemprop':'assigneeCurrent'})  
  patent_inventor = []
  patent_assignee = []

  for n in inventor:
      patent_inventor.append(n.get_text())

  df_patent_inventor = []
  for i in range(len(patent_inventor)):
    if i == 0:
      df_patent_inventor.append(patent_inventor[i])
    else :
      df_patent_inventor.append(' / '+patent_inventor[i])
  df_patent_inventor= "".join(df_patent_inventor)

  for n in assigneeCurrent:
      patent_assignee.append(n.get_text())
  df_patent_assignee = []
  for i in range(len(patent_assignee)):
    if i ==0:
      df_patent_assignee.append(patent_assignee[i])
    else :
      df_patent_assignee.append(' / '+patent_assignee[i])

  Str_assignee = "".join(df_patent_assignee)
  Str_assignee = Str_assignee.replace("\n","")
  Str_assignee = Str_assignee.lstrip()
  new_list_assignee = Str_assignee.split("   ")
  new_list_assignee= "".join(new_list_assignee)
  try:
    title = soup.select_one('body > search-app > article > h1').text
    st = title.find(' - ')
    end = title.rfind(' - ')
    df1 = pd.DataFrame(index=range(0,1), columns=['특허제목','특허코드','적용국가','Inventor','assignee'])
    df1['특허제목'] = title[st+3:end-9]
    df1['특허코드'] = title[:st]
    df1['적용국가'] = df_output
    df1['Inventor'] = df_patent_inventor
    df1['assignee'] = new_list_assignee
  except AttributeError:
    df1 = pd.DataFrame(index=range(0,1), columns=['특허제목','특허코드','적용국가','Inventor','assignee'])
    df1['특허제목'] = 'Withdrawn'
    df1['특허코드'] = url[-10:]
    df1['적용국가'] = '-'
    df1['Inventor'] = '-'
    df1['assignee'] = '-'
  return df1

def search_from_googlelinks(links):
    df2 = pd.DataFrame(columns=['특허제목','특허코드','적용국가','Inventor','assignee'])
    for link in links:
        df1 = application2(link)
        df2 = pd.concat([df2, df1], ignore_index=True)

    return df2
  
def Search_patent_news():
  #검색할 키워드 입력
  query = input('검색할 키워드를 입력하세요: ')


  #크롬드라이버로 원하는 url로 접속
  driver = webdriver.Chrome('./chromedriver')
  url = 'https://www.patentlyapple.com/patently-apple/'
  driver.get(url)
  time.sleep(2)

  #검색창에 키워드 입력 후 엔터

  search_box = driver.find_element_by_css_selector("#search-blog > div > input")
  search_box.send_keys(query)
  search_box.send_keys(Keys.RETURN)
  time.sleep(2)

  Apple_Patent_news = []
  Apple_Patent_link = []
  for k in range(0,100):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    Patent__apple_title = soup.find_all('h3', attrs={'class': 'entry-header font-entryheader'})
    for i in Patent__apple_title:
      Apple_Patent_news.append(i.get_text())

    Patent__apple_link = soup.select('h3 > a')
    for i in Patent__apple_link:
        Apple_Patent_link.append(i.attrs['href'])

    try:
      if(k==0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span').click()
      if(k > 0):
        driver.find_element_by_xpath('//*[@id="post-container"]/div/div[1]/div[1]/div/div/div[3]/div[11]/div/span[3]/a').click()
    except NoSuchElementException :
      print("Maximum page is ", k+1)
      break
  driver.close()

  df = pd.DataFrame(index=range(0,len(Apple_Patent_news)), columns=['News_Title','News_Link'])
  df['News_Title'] = Apple_Patent_news
  df['News_Link'] = Apple_Patent_link

  return df

def applepatent(url):
    # 제목과 링크 추출
    dflink = extractlink(url)
    
    # 필터들을 통해 얻어낸 코드리스트와 걸러지지않은 링크들
    list2 = filter1(dflink)
    
    # 코드 링크로의 구글 링크로의 변환
    links = applelink(list(set(list2)))
    
    # 구글에서 검색을 통한 데이터프레임생성
    df_output = search_from_googlelinks(links)
    
    return df_output
  
def applepatent_keyword():
    # 제목과 링크 추출
    dflink = Search_patent_news()
    
    # 필터들을 통해 얻어낸 코드리스트와 걸러지지않은 링크들
    list2 = filter1(dflink)
    
    # 코드 링크로의 구글 링크로의 변환
    links = applelink(list(set(list2)))
    
    # 구글에서 검색을 통한 데이터프레임생성
    df_output = search_from_googlelinks(links)
    
    return df_output

def df_to_db(keyword, df, client):
    items = df.to_dict("records")
    mydb = client['data']
    mycol = mydb[keyword]
    
    mycol.insert_many(items)

#how to use function1

key = 'stradvision, autonomous driving, CNN'

keywords, df = search(key,10)




#how to use function2

keyword = 'CNN,deep learning,autonomous driving'

inventor = '김계현'

assignee = 'stradvision'

country = 'kr'

df2 = detail_search(keyword, inventor, assignee, country)

how to use function3

url = 'https://www.patentlyapple.com/patently-apple/autonomous-vehicle-technology/'

df3 = applepatent(url)

for db

In [142]:
my_client = MongoClient("mongodb://localhost:27017/")

db에 에플을 저장

In [ ]:
url = 'https://www.patentlyapple.com/patently-apple/autonomous-vehicle-technology/'
df = applepatent(url)
df_to_db('autonomous driving1',df,my_client)

db에 키워드들을 저장
(리스트를 수정해서 돌리면 됨)

In [ ]:
keyword = 'autonomous driving'
inventor = ''
assignee_list = ['stradvision','Waymo LLC','Nvidia','argo AI','baidu','cruise','motional','Mobileye','Aurora','zoox','nuro','ford','intel','Hyundai Motor Company','volkswagen','yandex','daimler','bosch','aptiv','toyota','renault','nissan','mitsubishi motors']
country = '' 
for assignee in assignee_list:
    df = detail_search(keyword, inventor, assignee, country)
    df_to_db('autonomous driving1',df,my_client)


ford 중간에 끊김 (connection error)

그 이후로 실행


In [ ]:
keyword = 'autonomous driving'
inventor = ''
assignee_list = ['ford','intel','Hyundai Motor Company','volkswagen','yandex','daimler','bosch','aptiv','toyota','renault','nissan','mitsubishi motors']
country = '' 
for assignee in assignee_list:
    df = detail_search(keyword, inventor, assignee, country)
    df_to_db('autonomous driving1',df,my_client)

In [6]:
keyword = 'autonomous driving'
inventor = ''
assignee_list = ['stradvision']
country = '' 
for assignee in assignee_list:
    df = detail_search(keyword, inventor, assignee, country)
    df_to_db('autonomous driving_test',df,my_client)

C:\Users\junki\AppData\Local\Temp/ipykernel_35996/3201541700.py:315: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


데이터 필터링

In [175]:
df = pd.read_csv('autonomous driving.csv')

In [155]:
# df.pop('_id')
df.drop_duplicates(ignore_index=True)
df.to_csv('autonomous driving.csv', index=False)

In [176]:
display(df)

,Inventor,assignee,적용국가,특허제목,특허코드
0,James R. Wilson / Christopher D. Jones,Apple Inc,US(2019) / US(2019),Sensor enclosure and glass panels for sensor e...,US10852395B1
1,Gregory E. Rogers / Charles L. Greenlee,Apple Inc,US(2016) / US(2016),Ultrasonic sensor,US10871555B1
2,Peter F. Masschelein / Martin Melcher / Derek ...,Apple Inc,US(2020) / US(2018) / US(2020),Systems with windows,US11180005B2
3,-,-,-,Withdrawn,US00255002
4,Romain A. Teil / Jack J. WANDERMAN / Dominic P...,Apple Inc,US(2020) / US(2017) / US(2020),Cosmetic Integration of Displa,US20200214148A1
...,...,...,...,...,...
14431,NaN,Mitsubishi Motors Corp,JP,Motor-cooling device,JP2002204550A
14432,NaN,Mitsubishi Motors Corp,JP,Piston assembling device,JPH10138060A
14433,NaN,Mitsubishi Motors Corp,"EP, US, JP, KR, DE",Cylinder fuel injection type internal combusti...,JPH11294157A
14434,NaN,Mitsubishi Motors Corp,"US, KR, DE, SE",\n Device for a suction system as well as way...,SE510904C2


In [194]:
# df = change_ptname(df)
n = len(df.index)
print(n/100)
df2 = pd.DataFrame(columns=['특허제목','특허코드','적용국가','Inventor','assignee','google patent link','출원날짜','pdf link','인용횟수'])
for i in range(1,int(n/100) + 2):
    df_temp = df.loc[(i-1)*100:i*100-1]
    if i==int(n/100)+1:
        df_temp = df.loc[(i-1)*100:n]
    df_temp_new = change_ptname(df_temp)
    df2 = pd.concat([df2,df_temp_new], ignore_index=True)
    time.sleep(3)
    print(i)

144.36


C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:471: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['특허제목'] = whole_names
C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google patent link'] = google_links
C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

1
2
3
4
5
6
7
8
9
10
11


ConnectionError: HTTPSConnectionPool(host='patents.google.com', port=443): Max retries exceeded with url: /patent/US10826786B2/en (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001AF5122B610>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [217]:
# df = change_ptname(df)
n = len(df.index)
print(n/100)
# df2 = pd.DataFrame(columns=['특허제목','특허코드','적용국가','Inventor','assignee','google patent link','출원날짜','pdf link','인용횟수'])
# change here !!!
for i in range(128,int(n/100) + 2):
    df_temp = df.loc[(i-1)*100:i*100-1]
    if i==int(n/100)+1:
        df_temp = df.loc[(i-1)*100:n]
    df_temp_new = change_ptname(df_temp)
    df2 = pd.concat([df2,df_temp_new], ignore_index=True)
    time.sleep(3)
    print(i)

144.36


C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:471: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['특허제목'] = whole_names
C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['google patent link'] = google_links
C:\Users\junki\AppData\Local\Temp/ipykernel_41716/742750843.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


In [223]:
display(df2)

,특허제목,특허코드,적용국가,Inventor,assignee,google patent link,출원날짜,pdf link,인용횟수
0,Sensor enclosure and glass panels for sensor e...,US10852395B1,US(2019) / US(2019),James R. Wilson / Christopher D. Jones,Apple Inc,https://patents.google.com/patent/US10852395B1/en,2020-12-01,https://patentimages.storage.googleapis.com/54...,8
1,Ultrasonic sensor,US10871555B1,US(2016) / US(2016),Gregory E. Rogers / Charles L. Greenlee,Apple Inc,https://patents.google.com/patent/US10871555B1/en,2020-12-22,https://patentimages.storage.googleapis.com/0f...,24
2,Systems with windows,US11180005B2,US(2020) / US(2018) / US(2020),Peter F. Masschelein / Martin Melcher / Derek ...,Apple Inc,https://patents.google.com/patent/US11180005B2/en,2021-11-23,https://patentimages.storage.googleapis.com/4d...,13
3,,US00255002,-,-,-,https://patents.google.com/patent/US00255002/en,-,//www.google.com/,13
4,Cosmetic Integration of Displa,US20200214148A1,US(2020) / US(2017) / US(2020),Romain A. Teil / Jack J. WANDERMAN / Dominic P...,Apple Inc,https://patents.google.com/patent/US2020021414...,2020-07-02,https://patentimages.storage.googleapis.com/81...,13
...,...,...,...,...,...,...,...,...,...
13734,Motor-cooling device,JP2002204550A,JP,NaN,Mitsubishi Motors Corp,https://patents.google.com/patent/JP2002204550...,2002-07-19,/patent/JP2002204550A/ja,5
13735,Piston assembling device,JPH10138060A,JP,NaN,Mitsubishi Motors Corp,https://patents.google.com/patent/JPH10138060A/en,1998-05-26,/patent/JPH10138060A/ja,5
13736,Cylinder fuel injection type internal combusti...,JPH11294157A,"EP, US, JP, KR, DE",NaN,Mitsubishi Motors Corp,https://patents.google.com/patent/JPH11294157A/en,1999-10-26,/patent/JPH11294157A/ja,5
13737,\n Device for a suction system as well as way...,SE510904C2,"US, KR, DE, SE",NaN,Mitsubishi Motors Corp,https://patents.google.com/patent/SE510904C2/en,1999-07-05,/patent/SE510904C2/sv,5


In [222]:
df2 = df2.drop_duplicates(ignore_index=True)

돌려보고 안되면 i직접 바꿔가면서 15번 돌리기

In [220]:
df2.to_csv('autonomous_driving.csv')

In [224]:
df_to_db('autonomous driving refined(same name removed)',df2, my_client)